QIIME2(설치 Virtual Box)  
https://docs.google.com/presentation/d/1P6udhJDnvC2eurthDtHubcg4e_yDTsbsUNnecMM_gU8/edit?usp=sharing  
https://docs.qiime2.org/2019.1/ (tutorial list)

# QIIME2 Tutorial

## QIIME2 tutorial list
* “Moving Pictures” tutorial
* Fecal microbiota transplant (FMT) study: an exercise
* “Atacama soil microbiome” tutorial
* Differential abundance analysis with gneiss
* Importing data
* Exporting data
* Metadata in QIIME 2
* Filtering data
* Training feature classifiers with q2-feature-classifier
* Evaluating and controlling data quality with q2-quality-control
* Predicting sample metadata values with q2-sample-classifier
* Performing longitudinal and paired sample comparisons with q2-longitudinal
* Identifying and filtering chimeric feature sequences with q2-vsearch
* Alternative methods of read-joining in QIIME 2
* Clustering sequences into OTUs using q2-vsearch


## “Moving Pictures” tutorial
https://docs.qiime2.org/2019.1/tutorials/moving-pictures/ 
이번 분석은 5개의 시점을 두고 두 사람의 신체에서 추출한 신체 4곳의 미생물에 대해서 분석을 진행할 것이다.  
첫째로 antibiotic(항생물질)을 사용한다. 
  
## sample metadata
```$ mkdir Moving_Pictures```  
  
```$ cd Moving_Pictures```  
  
```(qiime2) $ wget -O "sample-metadata.tsv" "http://data.qiime2.org/2019.1/tutorials/moving-pictures/sample_metadata.tsv"```

## Obtaining and importing data  
  
```(qiime2) $ wget -O "emp-single-end-sequences/barcodes.fastq.gz" "https://data.qiime2.org/2019.1/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"```  

```(qiime2) $ wget -O "emp-single-end-sequences/sequence.fastq.gz" "https://data.qiime2.org/2019.1/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"```

  
semantic type QIIME 2는 EMPSingleEndSequence이다. 
EMPSingleEndSequence artifacts는 아직 sequence가 sample에 지정되지 않은  multiplexed한 sequence에 담겨있다.(따라서, sequence.fastq.gz 및 barcodes.fastq.gz 파일을 포함한다. barcodes.fastq.gz에는 sequence.fastq.gz의 각 sequence에 barcode가 들어있다.)
```(qiime2) $ qiime tools import --type EMPSingleEndSequences --input-path emp-single-end-sequences/ --output-path emp-single-end-sequences.qza```

## Demultiplexing sequences
demultiplex sequence는 각 sample 별로 관련된 barcode sequence를 아는 것이 필요하다. 
이 정보에는 sample metadata file이 담겨있다. 
command를 따라하면 sequences(demux emp-single command는 sequence에  barcode한 Earth Microbiome Project protocol 참조한다.)를 demultiplex할 수 있다.
demultiplexed sequences의 demux.qza를 QIIME2로 생성할 수 있다.  
  
```(qiime2) $ qiime demux emp-single --i-seqs emp-single-end-sequences.qza --m-barcodes-file sample-metadata.tsv --m-barcodes-column BarcodeSequence --o-per-sample-sequences demux.qza```
  
Demultiplexing 이후 생성된 파일을 이용하여 demultiplexing summary results를 생성한다.

```(qiime2) $ qiime demux summarize --i-data demux.qza --o-visualization demux.qzv```

* Visualization 

```(qiime2) $ qiime tools view demux.qzv```  
  
or https://view.qiime2.org/ 접속하여 파일을 드래그앤 드랍한다.  

예제) : https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fdemux.qzv 



## Sequence quality control and feature table construction
  
QIIME 2 plugins은 몇가지(DADA2, Deblur, basic quality-score-based filtering) 질좋은 control method 이용할 수 있다.  
Tutorial에서는 DADA2와 Deblur를 사용하며 원하는 것으로 변경하면서 사용해도 무방하다.


### Option 1: DADA2
DADA2는 Illumina amplicon sequence data를 detecting하고 correcting하는 pipeline이다.  
`q2-dada2` plugin을 통하여 사용이 가능하며, quality control process는 sequencing data에서 식별된 phiX판독 값을 추가적으로 filter한다.  
`*data2 denoise-single` method는 quality filtering에서 사용할 두가지 parameter를 요구한다.  
`--p-trim-left m`의 m에 시작하는 각 sequence 값을 넣고, `--p-trunc-len n`의 n에는 각 시퀀스를 자를 위치에 넣는다.   
위와 같이 pipeline을 통과하면 low quality sequences가 제거된다.



위와 같은 이유 때문에 m, n 값을 찾기 위해 `demux.qzv`파일로 생성된 qiime demux summarize의 Interactive Quality Plot tab을 확인해야한다.  
`demux.qzv`의 quality plots을 살펴보면 초기 sequence 에서는 quality가 좋은 상황을 볼 수 있으면 sequence가 후반으로 갈수록 quality가 떨어지는 것을 확인 할 수 있다.  
때문에 넉넉히 120 position  drop off할 것이다.  
최대 10분까지 걸리며 모든 과정중에 제일 오래 걸린다.

```(qiime2) $ qiime dada2 denoise-single --i-demultiplexed-seqs demux.qza --p-trim-left 0 --p-trunc-len 120 --o-representative-sequences rep-seqs-data2.qza --o-table table-data2.qza --o-denoising-stats stats-data2.qza```

#### dada2 Output visualization
```(qiime2) $ qiime metadata tabulate --m-input-file stats-data2.qza --o-visualization stats-dada2.qzv```

dada2로 Tutorial을 지속 진행한다면, 아래와 같이 변환해준다.  
`$ mv rep-seqs-data2.qza rep-seqs.qza`   
`$ mv table-data2.qza table.qza`



$$
\overrightarrow{y}\;=\;\overrightarrow{\beta_0}\;+\;\overrightarrow{\beta_{subject}}\;\overrightarrow{X_{subject}}\;+\;\overrightarrow{\beta_{sex}}\;\overrightarrow{X_{sex}}\;+\;\overrightarrow{\beta_{age}}\;\overrightarrow{X_{age}}\;+\; \overrightarrow{\beta_{sCD14ugml}}\;\overrightarrow{X_{sCD14ugml}}\;+\;\overrightarrow{\beta_{LBPugml}}\;\overrightarrow{X_{LBPugml}} 
$$

$$
b_i = \sqrt{\frac{rs}{r+s}} log\frac{g(x_r)}{g(x_s)}
$$

$$
d(x, y) \; = \; V[ln \frac{x}{y}]
$$

$$
\Delta Y_t = Y_t -Y_{t-1}
$$